In [ ]:
import os
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.tokenize import word_tokenize
from language_detector import detect_language
import pkg_resources
from symspellpy import SymSpell, Verbosity
import datetime
import time
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import numpy as np
warnings.filterwarnings('ignore')
from datetime import datetime
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import ast
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')


###### Preprocessing and Cleaning data #######

In [ ]:
meta = pd.read_csv("Loss_Narrative_data.csv")
meta.shape
print(meta.shape[0], " Total rows available")
count = 0
index = []
for i in range(len(meta)):
    #print(i)
    if type(meta.iloc[i, 2])== float or meta.iloc[i, 2]==0:
        count += 1
    else:
        index.append(i)
len(index)
print(len(index), "Total rows available")
dataset=meta.iloc[index,:]
dataset=dataset.reset_index()
dataset.drop("index", inplace = True, axis = 1)

dataset

sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
    pass
else:
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)


###################################
#### sentence level preprocess ####
###################################

# lowercase + base filter
# some basic normalization
def f_base(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    # normalization 1: xxxThis is a --> xxx. This is a (missing delimiter)
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', s)  # before lower case
    # normalization 2: lower case
    s = s.lower()
    # normalization 3: "&gt", "&lt"
    s = re.sub(r'&gt|&lt:', ' ', s)
    # normalization 4: letter repetition (if more than 2)
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # normalization 5: non-word repetition (if more than 1)
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # normalization 6: string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # normalization 7: stuff in parenthesis, assumed to be less informal
    s = re.sub(r'\(.*?\)', ' ', s)
    # normalization 8: xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # normalization 9: [.?!] --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    # normalization 10: ' ing ', noise text
    s = re.sub(r' ing ', ' ', s)
    # normalization 11: noise text
    s = re.sub(r'product received for free[.| ]', ' ', s)
    # normalization 12: phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)

    return s.strip()

#language detection
# def f_lan(s):
#     """
#     :param s: string to be processed
#     :return: boolean (s is English)
#     """

#     # some reviews are actually english but biased toward french
#     return detect_language(s) in {'English'}


###############################
#### word level preprocess ####
###############################

# filtering out punctuations and numbers
def f_punct(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with punct and number filter out
    """
    return [word for word in w_list if word.isalpha()]


# selecting nouns, verb and adjective 
# def f_noun(w_list):
#     """
#     :param w_list: word list to be processed
#     :return: w_list with only nouns selected
#     """
#     return [word for (word, pos) in nltk.pos_tag(w_list) 
#             if pos[:2] == 'NN' or pos=='JJ' or pos=="VBD"]


# typo correction
def f_typo(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with typo fixed by symspell. words with no match up will be dropped
    """
    w_list_fixed = []
    for word in w_list:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
        if suggestions:
            w_list_fixed.append(suggestions[0].term)
        else:
            w_list_fixed.append(word)
            # do word segmentation, deprecated for inefficiency
            # w_seg = sym_spell.word_segmentation(phrase=word)
            # w_list_fixed.extend(w_seg.corrected_string.split())
    return w_list_fixed


# stemming if doing word-wise
lemmatizer = WordNetLemmatizer()

def f_stem(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with stemming
    """
    
    return [lemmatizer.lemmatize(word ,'v') for word in w_list]

    
# filtering out stop words
# create English stop words list
import calendar
import datetime
# Month name from number
month_num = 1
month_abre = datetime.date(2015, month_num, 1).strftime('%b')
month_name = datetime.date(2015, month_num, 1).strftime('%B')

# Print list of all months from calendar
month_abr=[]
month=[]
for month_val in range(1, 13):
    month_abr.append(calendar.month_abbr[month_val])
    month.append(calendar.month_name[month_val])
month_abrev=[word.lower() for word in month_abr ] 
months=[word.lower() for word in month ]
months.extend(month_abrev)

stop_words = (list(
    set(get_stop_words('en'))
    |set(months)
))
# stop_words.extend(["claim",'id','dopa',"arise","claimant","allegations","auto","ploicy","care","work","potential",
#                    "service","allegdly","allege","clmt","insd",'insure','incident','covid-19','covid','complaint',
#                   'ovid','state','district','plaintiff','instal','due','demand','locate',
#                    'cause','client','customer','date','understand','app'])


def f_stopw(w_list):
    """
    filtering out stop words
    """
    return [word for word in w_list if word not in stop_words]

def cleanhtml(raw_html):
      cleanr = re.compile('<.*?>')
      cleantext = re.sub(cleanr, ' ', raw_html)
      return cleantext
  
def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw review texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    s= f_base(rw)
    s = re.sub(r"\bac\b","coolant",s)
    s = re.sub(r"\ba/c\b","coolant",s)
    s = re.sub(r"\btv\b","telivision",s)
    s = re.sub(r"\bair conditioning\b","coolant",s)
    s = re.sub(r"\bhard drive\b","disk",s)
    s = re.sub(r"\bpd\b","property damage",s)
    s = re.sub(r"\bbpp\b","business personal property",s)
    s = re.sub(r"\binj\b","injury",s)
    s = re.sub(r"\bbi\b","bodily injury",s)
    s = re.sub(r"\bdol\b","date of loss",s)
    s = re.sub(r"\bfcdpa\b","the fair debt collection practices act",s)
    s = re.sub(r"\bfdpca\b","the fair debt collection practices act",s)
    s = re.sub(r"\bfdcpa\b","the fair debt collection practices act",s)
    s = re.sub(r"\btcpa\b","telephone consumer protection act",s)
    s = re.sub(r"\bcpa\b","consumer protection act",s)
    s = re.sub(r"\bcccraa\b","consumer credit reporting agencies act",s)
    s = re.sub(r"\bhud\b","housing and urban development",s)
    s = re.sub(r"\bada\b","americans with disabilities act",s)
    s = re.sub(r"\bfcra\b","fair credit reporting act",s)
    s = re.sub(r"\bfcpa\b","The foreign corrupt practices act",s)
    s = re.sub(r"\bfha\b","fha fair housing act",s)
    s = re.sub(r"\bfacta\b","fair and accurate credit transaction Acts",s)
    s = re.sub(r"\ddtpa\b","deceptive trade practices act",s)
    s = re.sub(r"\beeoc\b","equal employment opportunity commission",s)
    s = re.sub(r"\bbrea\b","business research and economic advisors",s)
    s = re.sub(r"\bfdupta\b","florida deceptive and unfair trade practices act",s) 
    s = re.sub(r"\bwsba\b","washington state bar association",s)
    s = re.sub(r"\bfccpa\b","florida consumer collection practices act",s)
    s = re.sub(r"\bcfo\b","chief financial officer",s)
    s = re.sub(r"\bbacs\b","bankers automated clearing system",s)
    s=cleanhtml(s)
    s=s.replace('n/a'," ")
    s=s.replace('nbsp'," ")
    s=s.replace("dol"," ")
    if len(s)>0:
        if s[0]==" ":
            s=s[1:]
    if len(s)>0 :       
        if s[0].isdigit():
            s=s[2:]
    if len(s)>0:       
        if s[0]==" ":
            s=s[1:]
    if len(s)>0:
          if s[0]==".":
            s=s[1:]
    s=s.replace("’","'")
    s=re.sub("[^A-Za-z0-9 . ' $]"," ",s)
    s=re.sub("  +"," ",s)
    s=s.replace('..','.')   
    s=s.replace(" . ",". ")
    s=s.replace("clmt","claimant")
    s=s.replace("clmnt","claimant")
    s=s.replace("insd",'insured')
    s=s.replace('dmg',"damage")
    s=s.replace('dmgs',"damages")
#     if not f_lan(s):
#         return None
    return s


def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = f_punct(w_list)  
    w_list = f_typo(w_list)
    w_list= f_stem(w_list)
    #w_list = list(set(f_noun(w_list)))
    w_list = f_stopw(w_list)
    return w_list


def preprocess(docs):
    """
    Preprocess the data
    """
    print('Preprocessing raw texts ... ')
    n_docs = len(docs)
    sentences = []  # sentence level preprocessed
    token_lists = []  # word level preprocessed
    idx_in = [] # index of sample selected
    ref_id=[]
    groups=[]
    real_sen=[]
    #     samp = list(range(100))
    np.random.seed(1337)
    samp = list(range(len(docs)))
    for i, idx in enumerate(samp):
        sentence = preprocess_sent(docs[idx])
        token_list = preprocess_word(sentence)
        if token_list:
            idx_in.append(idx)
            sentences.append(sentence)
            token_lists.append(token_list)
            ref_id.append(data.iloc[i,0])
            groups.append(data.iloc[i,1])
            real_sen.append(cleanhtml(docs[idx]))
        print('{}--' .format(i), end='\r')
    print('Preprocessing raw texts. Done!')
    return real_sen,sentences, token_lists, ref_id, groups, idx_in
data = dataset 
rws = data.Text
real_sen,sentences, token_lists1, ref_id,groups,idx_in = preprocess(rws)
data_combined=pd.DataFrame({'Id':ref_id,'Grouping':groups,"Narration":real_sen,'Text':sentences,'Cleaned':token_lists1})
#data_combined.to_csv("Clusters_by_words/Cleaned_sentences_data.csv",index=False)

##### First party category and Third party category ######

In [ ]:
party_property=data_combined[(data_combined["Grouping"]=='1st Party Property') | (data_combined["Grouping"]=="3rd Party Property")]
party_property.reset_index(inplace=True,drop=True)
party_property.shape

In [ ]:
####################################################
####### Fire & Damage words occuring together ######
####################################################

clean_data=party_property["Cleaned"]
token_data=[]
for i in clean_data:
    token_data.append(" ".join( w for w in i))
fire1="fire damage"
val=[]
for idx,i in enumerate(token_data):
    if fire1 in i:
        val.append(idx)
fire1_data = party_property.iloc[val, :]
fire1_data["Label"]=list([7]*len(fire1_data))
new_data=party_property.drop(val)
new_data.reset_index(inplace=True,drop=True)     

####################################################
####### Water & Damage words occuring together #####
####################################################
clean_data=new_data["Cleaned"]
token_data=[]
for i in clean_data:
    token_data.append(" ".join( w for w in i))
water1="water damage"
val=[]
for idx,i in enumerate(token_data):
    if water1 in i:
        val.append(idx)
water1_data = new_data.iloc[val, :]
water1_data["Label"]=list([8]*len(water1_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)    

#####################################################
####### Floor & Damage words occuring together ######
#####################################################

clean_data=new_data["Cleaned"]
token_data=[]
for i in clean_data:
    token_data.append(" ".join( w for w in i))
floor1="floor damage"
val=[]
for idx,i in enumerate(token_data):
    if floor1 in i:
        val.append(idx)
floor1_data = new_data.iloc[val, :]
floor1_data["Label"]=list([10]*len(floor1_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)   

####################################################
##### Breah of Contract/Unsatissfactory work #######
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
breach=["breach","contract","agreement",'conflict',"unsatisfactorily","dissatisfy","incomplete"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in breach if x in i):
          val1.append(idx)
breach_data = new_data.iloc[val1, :]
breach_data["Label"]=list([1]*len(breach_data))
new_data=new_data.drop(val1)

####################################################
######## Copyright Trademark infringement ##########
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
trademark=["copyright","trademark","infringement"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in trademark if x in i):
          val1.append(idx)
trade_data = new_data.iloc[val1, :]
trade_data["Label"]=list([2]*len(trade_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################# Thef & Steal data ################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists) 
steal=["steal","theft","robbery","burglary","rob"]
val=[]
for idx,i in enumerate(token_lists):
      if any(True for x in steal if x in i):
          val.append(idx)
steal_data = new_data.iloc[val, :]
steal_data["Label"]=list([3]*len(steal_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)

####################################################
############### Missing & Lost data ################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists) 
steal=["miss","missing","lost"]
val=[]
for idx,i in enumerate(token_lists):
      if any(True for x in steal if x in i):
          val.append(idx)
steal2_data = new_data.iloc[val, :]
steal2_data["Label"]=list([4]*len(steal2_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)

####################################################
############# Equipment Damage data ################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
equip=["camera",'computer',"monitor","appliances","coolant","fridge","lamp","duct","stave","compressor","phones",
       "equipment","bag","laptop","photography","bulb","card","screen","phone",'eyeglasses',"microwave",
       ,"drone","backpack","headset","heaters","lens","ipad","stove","lamp","shutter","server"
       "lenses","tvs","television","tool","refrigerator","heater","machine","stool","machine","strap","woofer",
       "wire","cable","drop","trimmer","cameras","mac","laptops","electronic","pro","board","furnace",
       "battery","cornice","flash","printer","light","tripod","keyboard","tampon","scanner","valence","battery",
       "harddrive","mirror","nikon","tablet","photobooth","projector","display","garner","projectors","fibre",
       "speakers","photos","computers","cellphone","dryers","tablet","freezer","disk","blower","band","vacuum"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in equip if x in i):
          val1.append(idx)
equip_data = new_data.iloc[val1, :]
equip_data["Label"]=list([5]*len(equip_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############# Natural Disaster data ################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
natural=["hurricane","storm","windstorm","tornado","thunder","wildfire","wild","hail","rain"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in natural if x in i):
          val1.append(idx)
natural_data = new_data.iloc[val1, :]
natural_data["Label"]=list([6]*len(natural_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################# Fire Damage Data #################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
fire=["fire"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in fire if x in i):
          val1.append(idx)
fire_data = new_data.iloc[val1, :]
fire_data["Label"]=list([7]*len(fire_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################## Flood Damage Data ###############
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
water=["flood"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in water if x in i):
          val1.append(idx)
water_data2 = new_data.iloc[val1, :]
water_data2["Label"]=list([88]*len(water_data2))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################ Mold Damage Data ##################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
water=["mold","mould"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in water if x in i):
          val1.append(idx)
water_data3 = new_data.iloc[val1, :]
water_data3["Label"]=list([888]*len(water_data3))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############## Water Damage Data ###################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
water=["water","leak",'plumb',"line","puncture",
       "rain","supply","flood"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in water if x in i):
          val1.append(idx)
water_data = new_data.iloc[val1, :]
water_data["Label"]=list([8]*len(water_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############## Vehicle Damage Data #################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
veh=["vehicle","car","trailer","truck","auto","drive","truck","accident","cars","tractor","van","boat","bicycle"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in veh if x in i):
          val1.append(idx)
veh_data = new_data.iloc[val1, :]
veh_data["Label"]=list([9]*len(veh_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True) 

####################################################
########## Accidental Property damage Data #########
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
prop=["window","glass","home","break","house","lawn","clean","tree","damage","staircase","wheelchair","trimmer","sink","sewage",
       "windows","paint","floor","roof","ceiling","wall","mold","mould",'ceiling',"gazebo","driveway","mower","windshield",
       "carpet","gate","pool","tile","fence","door","doors","furniture","bathroom","motor","kitchen","tub","attic","pipe","property",
       "mirror","bathtub",'pipelines',"granite","construction","concrete","chair","char","furnace","ladder","dent","chemical"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in prop if x in i):
          val1.append(idx)
prop_data = new_data.iloc[val1, :]
prop_data["Label"]=list([10]*len(prop_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############### Data/Cyber Lost ####################
####################################################


clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
lostdata=["data","drive","disk","hack","hacking","website","software"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in lostdata if x in i):
          val1.append(idx)
lost_data = new_data.iloc[val1, :]
lost_data["Label"]=list([11]*len(lost_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################ Miscellaneous Data ################
####################################################

new_data['Label']=list([12]*len(new_data))

In [ ]:
Clustered_first=pd.concat([steal_data,steal2_data,equip_data,natural_data,fire1_data,fire_data,water1_data,water_data,water_data2,water_data3,
                           veh_data,prop_data,floor1_data,lost_data,breach_data,trade_data,new_data],axis=0)
Clustered_first.sort_values(by=["Label"],inplace=True)
Clustered_first.reset_index(inplace=True,drop=True)

In [ ]:
Clustered_first.to_csv("clustered property.csv",index=False)

###### General Liability Data #########

In [ ]:
General_liability=data_combined[(data_combined["Grouping"]=='GL')]
General_liability.reset_index(inplace=True,drop=True)
General_liability.shape

In [ ]:
####################################################
####### Fire & Damage words occuring together ######
####################################################

clean_data=General_liability["Cleaned"]
token_data=[]
for i in clean_data:
    token_data.append(" ".join( w for w in i))
fire1="fire damage"
val=[]
for idx,i in enumerate(token_data):
    if fire1 in i:
        val.append(idx)
fire1_data = General_liability.iloc[val, :]
fire1_data["Label"]=list([7]*len(fire1_data))
new_data=General_liability.drop(val)
new_data.reset_index(inplace=True,drop=True)  

####################################################
####### Water & Damage words occuring together #####
####################################################

clean_data=new_data["Cleaned"]
token_data=[]
for i in clean_data:
    token_data.append(" ".join( w for w in i))
water1="water damage"
val=[]
for idx,i in enumerate(token_data):
    if water1 in i:
        val.append(idx)
water1_data = new_data.iloc[val, :]
water1_data["Label"]=list([8]*len(water1_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)    

####################################################
####### Floor & Damage words occuring together #####
####################################################

clean_data=new_data["Cleaned"]
token_data=[]
for i in clean_data:
    token_data.append(" ".join( w for w in i))
floor1="floor damage"
val=[]
for idx,i in enumerate(token_data):
    if floor1 in i:
        val.append(idx)
floor1_data = new_data.iloc[val, :]
floor1_data["Label"]=list([10]*len(floor1_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)   

####################################################
##### Bodily & Injury words occuring together ######
####################################################

clean_data=new_data["Cleaned"]
token_data=[]
for i in clean_data:
    token_data.append(" ".join( w for w in i))
body="bodily injury"
val=[]
for idx,i in enumerate(token_data):
    if body in i:
        val.append(idx)
body_data = new_data.iloc[val, :]
body_data["Label"]=list([4]*len(body_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)   

####################################################
###### Property & Damage words occuring together ###
####################################################

clean_data=new_data["Cleaned"]
token_data=[]
for i in clean_data:
    token_data.append(" ".join( w for w in i))
prop1="property damage"
val=[]
for idx,i in enumerate(token_data):
    if prop1 in i:
        val.append(idx)
prop1_data = new_data.iloc[val, :]
prop1_data["Label"]=list([10]*len(prop1_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)   


####################################################
###### Breach of Contract/Unsatisfactory Work ######
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
breach=["breach","contract","agreement",'conflict',"unsatisfactorily","dissatisfy","incomplete"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in breach if x in i):
          val1.append(idx)
breach_data = new_data.iloc[val1, :]
breach_data["Label"]=list([1]*len(breach_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
######## Copyright Trademark infringement ##########
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
trademark=["copyright","trademark","infringement"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in trademark if x in i):
          val1.append(idx)
trade_data = new_data.iloc[val1, :]
trade_data["Label"]=list([2]*len(trade_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############## Theft and Steal Data ################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists) 
steal=["steal","theft","robbery","burglary","miss","missing","rob"]
val=[]
for idx,i in enumerate(token_lists):
      if any(True for x in steal if x in i):
          val.append(idx)
steal_data = new_data.iloc[val, :]
steal_data["Label"]=list([3]*len(steal_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)


####################################################
############# Bodily Injury Data ###################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
injury=["injury","injure","slip","trip",'thigh',"stairs","ankle","knee","infection","knee",
        "ankle","foot","facial","burns","burn","hurt","fracture","finger","fingers","patient",
        "allergic","eyebrow","skin","digest","bodily","skate","hair","dead",
        "hand","face","injuries","arm","wrist",'thighs','ear',"bodily","tooth","bi"
        "foot","death","dog","bite","leg","stair","head","barbeque","surgery","throat"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in injury if x in i):
          val1.append(idx)
injury_data = new_data.iloc[val1, :]
injury_data["Label"]=list([4]*len(injury_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################ Natural Disaster Data #############
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
natural=["flood","hurricane","storm","windstorm","tornado","thunder","wildfire","wind","hail","rain"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in natural if x in i):
          val1.append(idx)
natural_data = new_data.iloc[val1, :]
natural_data["Label"]=list([5]*len(natural_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)


####################################################
################ Equipment Damage Data #############
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
equip=["camera",'computer',"monitor","appliances","coolant","fridge","lamp","duct","stave","compressor","phones",
       "equipment","bag","laptop","photography","bulb","card","screen","phone",'eyeglasses',"microwave",
       "table","drone","backpack","headset","heaters","lens","ipad","stove","lamp","shutter","server"
       "lenses","tvs","television","tool","refrigerator","heater","machine","stool","strap","woofer",
       "wire","cable","drop","trimmer","cameras","mac","laptops","pro","board","furnace","telivisions"
      "battery","cornice","flash","printer","light","tripod","keyboard","tampon","scanner","valence","battery",
       "harddrive","mirror","nikon","tablet","photobooth","projector","display","garner","projectors","fibre",
      "speakers","photos","computers","cellphone","dryers","tablet","freezer","disk","blower","band","vacuum"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in equip if x in i):
          val1.append(idx)
equip_data = new_data.iloc[val1, :]
equip_data["Label"]=list([6]*len(equip_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################### Fire Damage Data ###############
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
fire=["fire"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in fire if x in i):
          val1.append(idx)
fire_data = new_data.iloc[val1, :]
fire_data["Label"]=list([7]*len(fire_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################ Water Damage Data #################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
water=["water","leak",'plumb',"line","rain","supply","puncture"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in water if x in i):
          val1.append(idx)
water_data = new_data.iloc[val1, :]
water_data["Label"]=list([8]*len(water_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################ Vehicle Damage Data ###############
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
veh=["vehicle","car","trailer","truck","auto","drive","truck","accident","cars","tractor","van","boat","bicycle"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in veh if x in i):
          val1.append(idx)
veh_data = new_data.iloc[val1, :]
veh_data["Label"]=list([9]*len(veh_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
######### Accidental Property Damage Data ##########
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
prop=["window","glass","home","break","house","lawn","clean","tree","damage","staircase","wheelchair","trimmer","sink","sewage",
      "windows","paint","floor","roof","ceiling","wall","mold","mould",'ceiling',"gazebo","driveway","mower","windshield"
      "carpet","gate","pool","tile","fence","door","doors","furniture","bathroom","motor","kitchen","tub","attic","pipe",
      "mirror","bathtub",'pipelines',"granite","construction","concrete","chair","char","furnace","ladder","dent","chemical",
      "sofa","couch","windshield","pot","stain","property","carpet","spill"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in prop if x in i):
          val1.append(idx)
prop_data = new_data.iloc[val1, :]
prop_data["Label"]=list([10]*len(prop_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############### Data/Cyber Lost ####################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
lostdata=["data","drive","disk","hack","hacking","website","software"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in lostdata if x in i):
          val1.append(idx)
lost_data = new_data.iloc[val1, :]
lost_data["Label"]=list([11]*len(lost_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
########### Fall/Slip Damage Data(BI) ###############
####################################################

injury_data.reset_index(inplace=True,drop=True)
clean_data=injury_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
fall=["fall","slip","trip",'fell']
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in fall if x in i):
          val1.append(idx)
fall_data1 = injury_data.iloc[val1, :]
fall_data1["Label"]=list([12]*len(fall_data1))
injury_data=injury_data.drop(val1)
injury_data.reset_index(inplace=True,drop=True)

####################################################
################### Fall Data ######################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
fall=["fall","step","steps","trip",'fell',"slip"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in fall if x in i):
          val1.append(idx)
fall_data = new_data.iloc[val1, :]
fall_data["Label"]=list([12]*len(fall_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
################ Unknown Data ######################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists) 
uk=["withdraw", "withdrew","incomplete"]
val0=[]
for idx,i in enumerate(token_lists):
      if any(True for x in uk if x in i):
          val0.append(idx)
uk_data = new_data.iloc[val0, :]
uk_data["Label"]=list(([13]*len(uk_data)))
new_data=new_data.drop(val0)
new_data.reset_index(inplace=True,drop=True)

####################################################
########### Miscellaneous Data #####################
####################################################

new_data['Label']=list([14]*len(new_data))

In [ ]:
GL_clustered=pd.concat([fire1_data,water1_data,floor1_data,body_data,prop1_data,breach_data,trade_data,steal_data,injury_data,natural_data,
equip_data,fire_data,water_data,veh_data,prop_data,lost_data,fall_data1,fall_data,uk_data,new_data],axis=0)
GL_clustered.sort_values(by=["Label"],inplace=True)
GL_clustered.reset_index(inplace=True,drop=True)
GL_clustered.shape

In [ ]:
GL_clustered.to_csv("GL_Clustered.csv",index=False)

###### Error And Omissions Data #######

In [ ]:
EO_data=data_combined[(data_combined["Grouping"]=='E&O')]
EO_data.reset_index(inplace=True,drop=True)
EO_data.shape

In [ ]:
####################################################
################## Unkown Data #####################
####################################################
new_data=EO_data
clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
      token_lists.append(i)
len(token_lists) 
wrng=["withdraw","duplicate","cancel","consolidate","urban"]
val=[]
for idx,i in enumerate(token_lists):
      if any(True for x in wrng if x in i):
          val.append(idx)
wrng_data = new_data.iloc[val, :]
wrng_data["Label"]=list([1]*len(wrng_data))
new_data=new_data.drop(val)
new_data.reset_index(inplace=True,drop=True)


####################################################
############## Discrimination/Abuse Data ###########
####################################################\

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
dis=["discrimination","rape","sexual","constitutional","urban",
     "racial","abuse","ethics","cha","harassment","equal","disabilities"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in dis if x in i):
          val1.append(idx)
dis_data = new_data.iloc[val1, :]
dis_data["Label"]=list([2]*len(dis_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)



####################################################
####### Copyright Trademark Infringement ###########
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
trademark=["copyright","trademark","infringement"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in trademark if x in i):
          val1.append(idx)
trade_data = new_data.iloc[val1, :]
trade_data["Label"]=list([3]*len(trade_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)


####################################################
################# Negligence Data ##################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
neg=["negligence","professional","negligent","service","tax"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in neg if x in i):
          val1.append(idx)
neg_data = new_data.iloc[val1, :]
neg_data["Label"]=list([4]*len(neg_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
####### Misrepresentation/Fraud Data ###############
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
mis=["misrepresentation","misrepresentations","fraud","fraudlent","disclosures","disclosure","trade",
     "disclose","scam","corruption","fraudulent","scammed","malpractice","corrupt","error","representation",
     "steal","theft","robbery","burglary","rob","lost","lose","discrepancy"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in mis if x in i):
          val1.append(idx)
mis_data = new_data.iloc[val1, :]
mis_data["Label"]=list([5]*len(mis_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
### Breach Of Contract/Unsatisfactory Work Data ####
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
breach=["breach","contract","agreement","incomplete","unsatisfactory","design","dissatisfy"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in breach if x in i):
          val1.append(idx)
breach_data = new_data.iloc[val1, :]
breach_data["Label"]=list([6]*len(breach_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)


####################################################
############# Lost/Cyber Data ######################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
lostdata=["data","photographs","photograph","software","hack","hacking","disk","website"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in lostdata if x in i):
          val1.append(idx)
lost_data = new_data.iloc[val1, :]
lost_data["Label"]=list([7]*len(lost_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############# Financial Damage Data ################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
money=["misappropriation","debts","debt","repossession","credit","banker","bankers","bank","telephone","foreclosure"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in money if x in i):
          val1.append(idx)
money_data = new_data.iloc[val1, :]
money_data["Label"]=list([8]*len(money_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############### Bodily Injury Data #################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
injury=["injury","injure","slip","trip",'thigh',"ankle","knee","bodily","burn","disability",
        "hand","face","injuries","arm","wrist",'thighs','ear',"bodily","tooth","bi","infection",
        "foot","dog","bite","death","leg","stair","head","surgery","throat","calf","facial"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in injury if x in i):
          val1.append(idx)
bi_data = new_data.iloc[val1, :]
bi_data["Label"]=list([9]*len(bi_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############# Equipment Damage Data ################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
token_lists[1]
equip=["camera",'computer',"monitor","appliances","coolant","fridge","duct","compressor","phones",
       "equipment","bag","laptop","photography","bulb","card","screen","phone",'eyeglasses',"microwave",
       "drone","backpack","headset","heaters","lens","ipad","stove","lamp","shutter","server","speakers",
       "lenses","tvs","television","tool","refrigerator","heater","machine","stool","machine","strap","woofer",
       "cable","cameras","mac","laptops","furnace",
       "cornice","flash","printer","microsoft","tripod","keyboard","tampon","scanner","battery",
       "harddrive","mirror","nikon","tablet","projector","display","garner","projectors",
       "speakers","photos","computers","cellphone","dryers","tablet","freezer","blower","vacuum","chandelier"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in equip if x in i):
          val1.append(idx)
equip_data = new_data.iloc[val1, :]
equip_data["Label"]=list([10]*len(equip_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############## Property Damage Data ################
####################################################

clean_data=new_data["Cleaned"]
token_lists=[]
for i in clean_data:
     token_lists.append(i)
len(token_lists)  
prop=["window","glass","home","break","house","lawn","clean","tree","bed","staircase","wheelchair","trimmer","sink","sewage",
       "windows","paint","floor","roof","ceiling","wall",'ceiling',"gazebo","driveway","mower","design","property","property",
       "carpet","gate","pool","tile","fence","door","doors","furniture","bathroom","motor","kitchen","tub","attic","pipe",
       "mirror","bathtub",'pipelines',"granite","construction","concrete","chair","char","furnace","ladder","dent","chemical",
        "rental","landlord","rent","apartment","mismanagement","estate","mold","mould","water","fire"]
val1=[]
for idx,i in enumerate(token_lists):
      if any(True for x in prop if x in i):
          val1.append(idx)
prop_data = new_data.iloc[val1, :]
prop_data["Label"]=list([11]*len(prop_data))
new_data=new_data.drop(val1)
new_data.reset_index(inplace=True,drop=True)

####################################################
############# Miscellaneous Data ###################
####################################################

new_data["Label"]=list([12]*len(new_data))


In [ ]:
EO_Clustered=pd.concat([wrng_data,dis_data,trade_data,neg_data,mis_data,
                          breach_data,lost_data,money_data,bi_data,equip_data,prop_data,new_data],axis=0)
EO_Clustered.sort_values(by=["Label"],inplace=True)
EO_Clustered.reset_index(inplace=True,drop=True)
EO_Clustered.shape

In [ ]:
EO_Clustered.to_csv("EO_Clustered.csv",index=False)